In [1]:
import pandas as pd
import numpy as np
# import cPickle 
import _pickle as cPickle
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()
import html.parser as html_parser

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [2]:
TEST_CASE = {}

In [6]:
import re
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()

import html.parser as html_parser

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

nltk.download("wordnet", "/Users/bharath/sourcecode/cynor/skill2vec/nltk_data/")
nltk.data.path.append('./nltk_data/')


import sys
if sys.version_info[0] >= 3:
    unicode = str

def skill_transform(skill, remove_stopwords = True):
    skill = unicode(skill)
    skill = html_parser.unescape(skill)
    
    skill = skill.replace("_", " ").split()
    skill = " ".join([sk for sk in skill if sk])
    
    skill = re.sub(r"\(.*\)", "", skill)
    skill = skill.replace("-", "") \
        .replace(".", "") \
        .replace(",", "") \
        .replace("-", "") \
        .replace(":", "") \
        .replace("(", "") \
        .replace(")", "") \
        .replace(u"åá", "") \
        .replace(u"&", "and") \
        .replace(" js", "js") \
        .replace("-js", "js") \
        .replace("_js", "js") \
        .replace("java script", "js") 
    
    skill = skill.lower()
    
    # Special cases replace
    special_case = {}
    special_case["javascript"] = [ "js", "java script", "javascripts", "java scrip" ]
    special_case["wireframe"] = [ "wireframes", "wire frame", "wire frames", "wire-frame", "wirefram", "wire fram", "wireframing" ]
    special_case["OOP"] = [  "object oriented", "object oriented programming", ]
    special_case["OOD"] = [ "object oriented design", ]
    special_case["OLAP"] = [ "online analytical processing",  ]
    special_case["Ecommerce"] = [ "e commerce",  ]
    special_case["consultant"] = [ "consulting",  ]
    special_case["ux"] = [ "user experience", "web user experience design", "user experience design", "ux designer", "user experience/ux" ]
    special_case["html5"] = [ "html 5",  ]
    special_case["j2ee"] = [ "jee",  ]
    special_case["osx"] = [ "mac os x", "os x" ]
    special_case["senior"] = [ "sr" ]
    special_case["qa"] = [ "quality",  ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["webservice"] = [ "webservices", "website", "webapps" ]
    special_case["xml"] = [ "xml file", "xml schemas", "xml/json", "xml web service" ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["nlp"] = [ "natural language process", "natural language", "nltk" ]
    special_case["aws"] = [ "amazon web service", ]
    
    for root_skill in special_case:
        if skill in special_case[root_skill]:
            skill = root_skill
    
    # Special case regex
    special_case_regex = {
        r'^angular.*$': 'angularjs',
        r'^node.*$': 'nodejs',
        r'^(.*)[_\s]js$': '\\1js',
        r'^(.*) js$': '\\1js',
        r'^(.*) (and|or).*$': '\\1',
    }
    for regex_rule in special_case_regex:
        after_skill = re.sub(regex_rule, special_case_regex[regex_rule], skill)
        if after_skill != skill:
            skill = after_skill
            break
    
    # Stem
    if len(skill) > 2:
        skill_after = skill.split(" ")
        skill_after = [wordnet_lemmatizer.lemmatize(sk, pos="v") for sk in skill_after]
        skill_after = " ".join(skill_after)
        skill = skill_after
    
    # skill stopwords 
    if remove_stopwords:
        skill_stopwords = [ "app", "touch", "the", "application", "programming", "program", "design"
                           "developer", "framework", "development", "programmer", "technologies",
                          "advance", "core"]
        skill_after = skill.split(" ")
        skill = " ".join([ sk for sk in skill_after if sk not in skill_stopwords ])
    
    skill = skill.lower().strip().replace(" ", "_")
    skill = re.sub(' +',' ', skill)
    
    # NOTE: replace js tail
    skill = re.sub('js$','', skill)
    
    return skill

skill_transform("js")
skill_transform("angularjs")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bharath/sourcecode/cynor/skill2vec/nltk_data/..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


'angular'

In [7]:
skill_transform("html5_and_css3_or_html6")

'html5_and_css3'

In [8]:
def test_case(model, model_name):
    cases = [ "machine learning", "js", "javascript", "python", "html", "html5", "css", "angular", "nodejs" ]
    TEST_CASE[model_name] = []
    for case in cases:
        try:
            TEST_CASE[model_name].append(model.similar_by_word(skill_transform(case)))
        except:
            pass

In [34]:
# data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
data_must_have = pd.read_csv('skill2vec_50K.csv', header = 0, encoding='ISO-8859-1', low_memory=False)
data_must_have.drop_duplicates(inplace=True, keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
del data_must_have["job_brief_id"]
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)

unique_skill_before = len(data_must_have['keyword_name'].unique())
print("Before: %s" % unique_skill_before)

data_must_have['skill'] = data_must_have['keyword_name'].apply(skill_transform)

unique_skill_after = len(data_must_have['skill'].unique())
print("After: %s" % unique_skill_after)

print("Resize: %s%%" % (unique_skill_after * 100.0 / unique_skill_before))

# data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
# data_must_have

KeyError: 'job_title'

In [13]:
# pd.set_option('display.max_rows', 1500)
# data_must_have.drop_duplicates(subset=["keyword_name", "skill"])[["keyword_name", "skill"]].sort_values("skill")[3000:]
musthave_data = data_must_have.groupby('job_title')['skill'].apply(list).tolist()

print ("Training model...")
musthave_model = word2vec.Word2Vec(musthave_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_model.init_sims(replace=True)
test_case(musthave_model, "musthave_model")

2021-09-30 22:55:05,785 : INFO : collecting all words and their counts
2021-09-30 22:55:05,786 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 22:55:05,831 : INFO : collected 6198 word types from a corpus of 80810 raw words and 5649 sentences
2021-09-30 22:55:05,833 : INFO : Creating a fresh vocabulary


Training model...


2021-09-30 22:55:05,993 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 6198 unique words (100.0%% of original 6198, drops 0)', 'datetime': '2021-09-30T22:55:05.915065', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-09-30 22:55:05,996 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 80810 word corpus (100.0%% of original 80810, drops 0)', 'datetime': '2021-09-30T22:55:05.996503', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-09-30 22:55:06,109 : INFO : deleting the raw counts dictionary of 6198 items
2021-09-30 22:55:06,111 : INFO : sample=0.001 downsamples 35 most-common words
2021-09-30 22:55:06,113 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 77128.92432672551 word corpus 

In [16]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
dice_data = [ [ skill_transform(d.replace("_", " ")) for d in dice if d is not None ] for dice in data_dice if dice is not None ]


musthave_dice_data = dice_data + musthave_data
musthave_dice_data = [ list(set([ d for d in data if d is not None and len(d) > 0 ])) for data in musthave_dice_data  ]

print(len(musthave_dice_data))
musthave_dice_data

print ("Training model...")
musthave_dice_model = word2vec.Word2Vec(musthave_dice_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_model.init_sims(replace=True)
test_case(musthave_dice_model, "musthave_dice_model")

2021-09-30 22:56:34,243 : INFO : collecting all words and their counts
2021-09-30 22:56:34,244 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 22:56:34,369 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2021-09-30 22:56:34,386 : INFO : collected 12535 word types from a corpus of 254463 raw words and 12795 sentences
2021-09-30 22:56:34,387 : INFO : Creating a fresh vocabulary


12795
Training model...


2021-09-30 22:56:34,501 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 12535 unique words (100.0%% of original 12535, drops 0)', 'datetime': '2021-09-30T22:56:34.501490', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-09-30 22:56:34,503 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 254463 word corpus (100.0%% of original 254463, drops 0)', 'datetime': '2021-09-30T22:56:34.503550', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-09-30 22:56:34,673 : INFO : deleting the raw counts dictionary of 12535 items
2021-09-30 22:56:34,675 : INFO : sample=0.001 downsamples 1 most-common words
2021-09-30 22:56:34,679 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 254448.57313022885 word co

In [65]:
# model.similar_by_word(skill_transform('machine learning'))

In [66]:
# model.similar_by_word('python')

In [18]:
musthave_dice_naruki_data = musthave_dice_data
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')

data_naruki['skill'] = data_naruki['skill'].apply(skill_transform)
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print(len(data_naruki_final))

for skills in data_naruki_final:
    if len(skills) > 2 and skills not in musthave_dice_naruki_data:
        musthave_dice_naruki_data.append(skills)
        
len(musthave_dice_naruki_data)

120107


110659

In [20]:
# import cPickle as pickle
import _pickle as pickle

pickle.dump( musthave_dice_naruki_data, open( "cynor_model.p", "wb" ) )

In [21]:
print ("Training model...")
musthave_dice_naruki_model = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=3, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_naruki_model.init_sims(replace=True)
test_case(musthave_dice_naruki_model, "musthave_dice_naruki_model")

2021-09-30 23:18:20,709 : INFO : collecting all words and their counts
2021-09-30 23:18:20,713 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 23:18:20,801 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2021-09-30 23:18:20,872 : INFO : PROGRESS: at sentence #20000, processed 316511 words, keeping 21557 word types
2021-09-30 23:18:20,932 : INFO : PROGRESS: at sentence #30000, processed 403013 words, keeping 30521 word types


Training model...


2021-09-30 23:18:20,971 : INFO : PROGRESS: at sentence #40000, processed 489608 words, keeping 38336 word types
2021-09-30 23:18:21,048 : INFO : PROGRESS: at sentence #50000, processed 579752 words, keeping 45956 word types
2021-09-30 23:18:21,082 : INFO : PROGRESS: at sentence #60000, processed 666563 words, keeping 51999 word types
2021-09-30 23:18:21,121 : INFO : PROGRESS: at sentence #70000, processed 751299 words, keeping 58854 word types
2021-09-30 23:18:21,174 : INFO : PROGRESS: at sentence #80000, processed 836204 words, keeping 65148 word types
2021-09-30 23:18:21,215 : INFO : PROGRESS: at sentence #90000, processed 923057 words, keeping 70252 word types
2021-09-30 23:18:21,258 : INFO : PROGRESS: at sentence #100000, processed 1013119 words, keeping 74894 word types
2021-09-30 23:18:21,305 : INFO : PROGRESS: at sentence #110000, processed 1099321 words, keeping 80582 word types
2021-09-30 23:18:21,309 : INFO : collected 80689 word types from a corpus of 1105245 raw words and 1

2021-09-30 23:18:51,131 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-09-30 23:18:51,186 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-09-30 23:18:51,189 : INFO : EPOCH - 5 : training on 1105245 raw words (1014409 effective words) took 6.7s, 150323 effective words/s
2021-09-30 23:18:51,191 : INFO : Word2Vec lifecycle event {'msg': 'training on 5526225 raw words (5072595 effective words) took 28.8s, 175909 effective words/s', 'datetime': '2021-09-30T23:18:51.191140', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
2021-09-30 23:18:51,192 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=23482, vector_size=100, alpha=0.025)', 'datetime': '2021-09-30T23:18:51.192202', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
/tm

In [22]:
TEST_CASE

{'musthave_model': [],
 'musthave_dice_model': [],
 'musthave_dice_naruki_model': []}

In [23]:
TEST_CASE_TABLE = [ pd.DataFrame(TEST_CASE[t]) for t in TEST_CASE ]

In [24]:
[ t for t in TEST_CASE ]

['musthave_model', 'musthave_dice_model', 'musthave_dice_naruki_model']

In [25]:
TEST_CASE_TABLE[0]

""


In [73]:
TEST_CASE_TABLE[1]

,0,1,2,3,4,5,6,7,8,9
0,"(data_mine, 0.946587741375)","(log_mine, 0.913221418858)","(parallel, 0.91216814518)","(algorithms, 0.911779403687)","(anomaly_detection, 0.907564878464)","(large_scale, 0.894648551941)","(statistics, 0.89054453373)","(odesign, 0.887861430645)","(mapreduce, 0.87881141901)","(recommendation, 0.870700955391)"
1,"(drupal, 0.966448664665)","(jquery, 0.965409040451)","(less, 0.965108454227)","(behaviour_drive, 0.965004563332)","(bdd, 0.964688897133)","(sass, 0.964326322079)","(bootstrap, 0.963802874088)","(grunt, 0.962328135967)","(jasmine, 0.96202725172)","(mvw, 0.959979176521)"
2,"(drupal, 0.966448664665)","(jquery, 0.965409040451)","(less, 0.965108454227)","(behaviour_drive, 0.965004563332)","(bdd, 0.964688897133)","(sass, 0.964326322079)","(bootstrap, 0.963802874088)","(grunt, 0.962328135967)","(jasmine, 0.96202725172)","(mvw, 0.959979176521)"
3,"(ruby_on_rail, 0.944096446037)","(ror, 0.944012284279)","(engineer_in_test, 0.943315684795)","(blackbox_test, 0.928975045681)","(80211, 0.92416870594)","(ieee_80211, 0.923894822598)","(omnipeek, 0.923828601837)","(https, 0.92302942276)","(single_player, 0.922428548336)","(iperf, 0.92227178812)"
4,"(css, 0.972033441067)","(template, 0.968834638596)","(adobe_experience_management_suite, 0.96435648...","(cq, 0.964027941227)","(css3, 0.963955402374)","(xhtml, 0.962866663933)","(adobe_cq, 0.958325028419)","(aem, 0.956038534641)","(foundation, 0.955864429474)","(css2, 0.955240547657)"
5,"(css3, 0.970298290253)","(aem, 0.953352451324)","(jquery, 0.953019797802)","(less, 0.945244014263)","(css, 0.941391468048)","(adobe_experience_management_suite, 0.94047647...","(template, 0.940237283707)","(angular, 0.939173579216)","(html, 0.937180519104)","(cq, 0.934721469879)"
6,"(html, 0.972033500671)","(jquery, 0.949057996273)","(javascript, 0.94364207983)","(html5, 0.941391468048)","(css3, 0.93936920166)","(template, 0.939226865768)","(adobe_experience_management_suite, 0.9372433424)","(aem, 0.936206579208)","(cq, 0.935415387154)","(sass, 0.935126662254)"
7,"(knockout, 0.993182778358)","(backbone, 0.992327332497)","(ember, 0.989857435226)","(jquerry, 0.988534092903)","(spa, 0.987402558327)","(durandal, 0.986958682537)","(html/css, 0.98613679409)","(three, 0.985328137875)","(foundation, 0.983917832375)","(npm, 0.98370295763)"
8,"(ember, 0.977777481079)","(etl, 0.976830482483)","(webgl, 0.976110219955)","(svg, 0.975852310658)","(responsive/adaptive_design, 0.973373353481)","(spa, 0.972387611866)","(backbone, 0.971573591232)","(npm, 0.971285581589)","(extract, 0.970165967941)","(three, 0.969876766205)"


In [74]:
TEST_CASE_TABLE[2]

,0,1,2,3,4,5,6,7,8,9
0,"(data_scientist, 0.907000541687)","(nlp, 0.893116116524)","(anomaly_detection, 0.871943593025)","(deep_learn, 0.870284318924)","(random_forest, 0.862106204033)","(recommendation, 0.862098574638)","(scala_language, 0.857527852058)","(k__mean, 0.854613780975)","(tensorflow, 0.853815555573)","(time_series_analysis, 0.85380089283)"
1,"(html5, 0.846334934235)","(ajax, 0.837237775326)","(css3, 0.83621609211)","(mvc_architecture, 0.816482722759)","(mvc_frameworks, 0.808582544327)","(web_tool, 0.807993650436)","(oops_concept, 0.806290745735)","(bootstrap, 0.802878439426)","(senior_web_developer, 0.79932641983)","(webdeveloper, 0.797972679138)"
2,"(html5, 0.846334934235)","(ajax, 0.837237775326)","(css3, 0.83621609211)","(mvc_architecture, 0.816482722759)","(mvc_frameworks, 0.808582544327)","(web_tool, 0.807993650436)","(oops_concept, 0.806290745735)","(bootstrap, 0.802878439426)","(senior_web_developer, 0.79932641983)","(webdeveloper, 0.797972679138)"
3,"(ruby, 0.771286308765)","(perl, 0.757723748684)","(relational_databases, 0.739183008671)","(test_frameworks, 0.734647274017)","(map_reduce, 0.731255888939)","(devop, 0.72961807251)","(kinesis, 0.729043483734)","(bigquery, 0.727345228195)","(administrations, 0.72573363781)","(python_programming/scripting_language, 0.7253..."
4,"(xhtml, 0.79220533371)","(html5, 0.790575385094)","(javascript, 0.777606129646)","(web_design, 0.771491289139)","(ajax, 0.769158482552)","(css3, 0.767653107643)","(html_xhtml, 0.756079077721)","(my_sql, 0.754533052444)","(web_tool, 0.753923177719)","(dreamweaver, 0.749688744545)"
5,"(css3, 0.937505722046)","(ui_developer, 0.92447155714)","(bootstrap, 0.923129379749)","(front__end, 0.915638208389)","(css_3, 0.913625717163)","(ui_engineer, 0.906963944435)","(responsive, 0.905551493168)","(html4, 0.904073417187)","(ui_lead, 0.903929889202)","(front_end_engineer, 0.903929233551)"
6,"(html5, 0.856252551079)","(css3, 0.811155676842)","(web_tool, 0.809652686119)","(ui_developer, 0.807193517685)","(bootstrap, 0.805215239525)","(senior_web_developer, 0.802639603615)","(mvc_frameworks, 0.799567103386)","(css_3, 0.795899987221)","(webdeveloper, 0.795441031456)","(ui_developers, 0.792991220951)"
7,"(node, 0.921982288361)","(bootstrap, 0.921577572823)","(backbone, 0.894273757935)","(react, 0.881976783276)","(ui_developer, 0.872991323471)","(html5, 0.869287610054)","(knockout, 0.868739485741)","(javascript_developer, 0.866702198982)","(bower, 0.86581158638)","(full_stack_developer, 0.865062236786)"
8,"(react, 0.938898682594)","(backbone, 0.924705505371)","(mean_stack, 0.924305737019)","(angular, 0.921982288361)","(mean, 0.909098386765)","(bower, 0.905931711197)","(full_stack_developer, 0.902433216572)","(npm, 0.901626110077)","(frontend, 0.901094675064)","(grunt, 0.898154914379)"


In [26]:
musthave_dice_naruki_model.wv.save_word2vec_format("musthave_dice_naruki.model")

2021-09-30 23:19:39,867 : INFO : storing 23482x100 projection weights into musthave_dice_naruki.model


In [27]:
!ls -l

total 253176
drwxr-xr-x  12 bharath  staff       384 Sep 30 22:50 .git
-rw-r--r--   1 bharath  staff      1045 Sep 30 21:56 .gitignore
drwxr-xr-x   9 bharath  staff       288 Sep 30 22:57 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff      1069 Sep 30 21:56 LICENSE
-rw-r--r--   1 bharath  staff        42 Sep 30 21:56 README.md
-rw-r--r--   1 root     staff  15150315 Sep 30 23:18 cynor_model.p
-rw-r--r--   1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  28303231 Sep 30 23:19 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  17778882 Sep 30 21:56 musthave_dice_naruki_100d.model
-rw-r--r--   1 bharath  staff  35249875 Sep 30 21:56 musthave_dice_naruki_200d.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_full
drwxr-xr-x   3 bharath  staff        96 Sep 30 22:52 nltk_data
-rw-r--r--   1 bharath  staff   3972850 Sep 30 21:56 prep_data_tokens_underscore_1
-rw-r--r--   1 bharath  staff   1276232 Sep 30 21:56 prepare_

In [28]:
musthave_dice_naruki_model_100d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=100, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_naruki_model_200d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=200, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

2021-09-30 23:19:57,599 : INFO : collecting all words and their counts
2021-09-30 23:19:57,604 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 23:19:57,715 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2021-09-30 23:19:57,781 : INFO : PROGRESS: at sentence #20000, processed 316511 words, keeping 21557 word types
2021-09-30 23:19:57,826 : INFO : PROGRESS: at sentence #30000, processed 403013 words, keeping 30521 word types
2021-09-30 23:19:57,877 : INFO : PROGRESS: at sentence #40000, processed 489608 words, keeping 38336 word types
2021-09-30 23:19:57,912 : INFO : PROGRESS: at sentence #50000, processed 579752 words, keeping 45956 word types
2021-09-30 23:19:57,962 : INFO : PROGRESS: at sentence #60000, processed 666563 words, keeping 51999 word types
2021-09-30 23:19:58,026 : INFO : PROGRESS: at sentence #70000, processed 751299 words, keeping 58854 word types
2021-09-30 23:19:58,073 : INFO : PROGRESS: at 

2021-09-30 23:20:25,072 : INFO : EPOCH 5 - PROGRESS: at 65.17% examples, 159038 words/s, in_qsize 7, out_qsize 0
2021-09-30 23:20:26,081 : INFO : EPOCH 5 - PROGRESS: at 97.42% examples, 179594 words/s, in_qsize 3, out_qsize 1
2021-09-30 23:20:26,082 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-09-30 23:20:26,170 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-09-30 23:20:26,180 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-09-30 23:20:26,210 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-09-30 23:20:26,213 : INFO : EPOCH - 5 : training on 1105245 raw words (992323 effective words) took 5.5s, 179457 effective words/s
2021-09-30 23:20:26,217 : INFO : Word2Vec lifecycle event {'msg': 'training on 5526225 raw words (4961805 effective words) took 27.4s, 181377 effective words/s', 'datetime': '2021-09-30T23:20:26.217497', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) 

2021-09-30 23:20:45,050 : INFO : EPOCH 4 - PROGRESS: at 4.55% examples, 107911 words/s, in_qsize 7, out_qsize 2
2021-09-30 23:20:46,100 : INFO : EPOCH 4 - PROGRESS: at 27.80% examples, 171214 words/s, in_qsize 7, out_qsize 0
2021-09-30 23:20:47,143 : INFO : EPOCH 4 - PROGRESS: at 48.30% examples, 170192 words/s, in_qsize 7, out_qsize 0
2021-09-30 23:20:48,192 : INFO : EPOCH 4 - PROGRESS: at 65.13% examples, 161134 words/s, in_qsize 8, out_qsize 0
2021-09-30 23:20:49,226 : INFO : EPOCH 4 - PROGRESS: at 94.22% examples, 175853 words/s, in_qsize 6, out_qsize 0
2021-09-30 23:20:49,308 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-09-30 23:20:49,386 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-09-30 23:20:49,419 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-09-30 23:20:49,449 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-09-30 23:20:49,450 : INFO : EPOCH - 4 : training on 1105245 raw wor

In [29]:
musthave_dice_naruki_model_100d.wv.save_word2vec_format("musthave_dice_naruki_100d.model")
musthave_dice_naruki_model_200d.wv.save_word2vec_format("musthave_dice_naruki_200d.model")

2021-09-30 23:20:53,632 : INFO : storing 17150x100 projection weights into musthave_dice_naruki_100d.model
2021-09-30 23:20:56,040 : INFO : storing 17150x100 projection weights into musthave_dice_naruki_200d.model


In [30]:
!ls -l
!free -m

total 227872
drwxr-xr-x  12 bharath  staff       384 Sep 30 22:50 .git
-rw-r--r--   1 bharath  staff      1045 Sep 30 21:56 .gitignore
drwxr-xr-x   9 bharath  staff       288 Sep 30 22:57 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff      1069 Sep 30 21:56 LICENSE
-rw-r--r--   1 bharath  staff        42 Sep 30 21:56 README.md
-rw-r--r--   1 root     staff  15150315 Sep 30 23:18 cynor_model.p
-rw-r--r--   1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  28303231 Sep 30 23:19 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  20043489 Sep 30 23:20 musthave_dice_naruki_100d.model
-rw-r--r--   1 bharath  staff  20049273 Sep 30 23:20 musthave_dice_naruki_200d.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_full
drwxr-xr-x   3 bharath  staff        96 Sep 30 22:52 nltk_data
-rw-r--r--   1 bharath  staff   3972850 Sep 30 21:56 prep_data_tokens_underscore_1
-rw-r--r--   1 bharath  staff   1276232 Sep 30 21:56 prepare_

In [31]:
musthave_dice_naruki_model.wv.similar_by_word(skill_transform('javascript'),topn=10)



[('ajax', 0.813778281211853),
 ('bootstrap', 0.8119865655899048),
 ('css3', 0.8106129169464111),
 ('jquery', 0.8005176186561584),
 ('html5', 0.8004119396209717),
 ('web_tool', 0.7980742454528809),
 ('oops_concept', 0.796851396560669),
 ('mvc_architecture', 0.7949170470237732),
 ('web_developement', 0.7898707389831543),
 ('mvc_frameworks', 0.7897635698318481)]

In [32]:
musthave_dice_naruki_model_200d.wv.similar_by_word(skill_transform('html5'))

[('css3', 0.9009639620780945),
 ('ui_developer', 0.8738539218902588),
 ('css_3', 0.8734769225120544),
 ('bootstrap', 0.8689416646957397),
 ('html4', 0.8646476864814758),
 ('front__end', 0.8634669184684753),
 ('responsive_web', 0.8613139390945435),
 ('javascript_developer', 0.8600084781646729),
 ('html_developer', 0.8592700362205505),
 ('ui_engineer', 0.8586831092834473)]

In [82]:
# [(u'css3', 0.9307963252067566),
#  (u'bootstrap', 0.9234713912010193),
#  (u'ui_developer', 0.9192823171615601),
#  (u'front_end_engineer', 0.9077576398849487),
#  (u'front__end', 0.9058327674865723),
#  (u'css_3', 0.9039251208305359),
#  (u'ui_engineer', 0.9014566540718079),
#  (u'adobe_experience_management_suite', 0.9006956815719604),
#  (u'html4', 0.9003456234931946),
#  (u'user_interface_developer', 0.8981920480728149)]

[(u'visual_design', 0.8798125982284546),
 (u'user_interface_design', 0.8789986371994019),
 (u'ux_design', 0.8746244311332703),
 (u'ui', 0.8447457551956177),
 (u'interaction_designer', 0.8409931063652039),
 (u'interaction_design', 0.8397772908210754),
 (u'user_interaction', 0.8348618745803833),
 (u'wireframe', 0.8340160250663757),
 (u'transform', 0.8149587512016296),
 (u'mobile_design', 0.8148149251937866)]

In [83]:
musthave_dice_naruki_model.similar_by_word(skill_transform('javascript'))

[(u'html5', 0.8463349342346191),
 (u'ajax', 0.8372377753257751),
 (u'css3', 0.8362160921096802),
 (u'mvc_architecture', 0.8164827227592468),
 (u'mvc_frameworks', 0.8085825443267822),
 (u'web_tool', 0.8079936504364014),
 (u'oops_concept', 0.8062907457351685),
 (u'bootstrap', 0.8028784394264221),
 (u'senior_web_developer', 0.7993264198303223),
 (u'webdeveloper', 0.7979726791381836)]

In [84]:
musthave_dice_naruki_model.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9388986825942993),
 (u'backbone', 0.9247055053710938),
 (u'mean_stack', 0.9243057370185852),
 (u'angular', 0.9219822883605957),
 (u'mean', 0.9090983867645264),
 (u'bower', 0.9059317111968994),
 (u'full_stack_developer', 0.9024332165718079),
 (u'npm', 0.9016261100769043),
 (u'frontend', 0.9010946750640869),
 (u'grunt', 0.8981549143791199),
 (u'css/javascript', 0.8971441984176636),
 (u'gulp', 0.8954569697380066),
 (u'bootstrap', 0.8946527242660522),
 (u'mongodb', 0.894255518913269),
 (u'javascript_developer', 0.8926291465759277),
 (u'mean_stack_developer', 0.8926271200180054),
 (u'mongo_db', 0.8904428482055664),
 (u'front_end_web_developer', 0.8894308805465698),
 (u'service_workers', 0.8887876272201538),
 (u'ui_architect', 0.8884862661361694),
 (u'less', 0.888136088848114),
 (u'loopback', 0.8872900009155273),
 (u'firebase', 0.8807092905044556),
 (u'mocha', 0.8802285194396973),
 (u'meanstack', 0.880145788192749),
 (u'front__end', 0.8791353106498718),
 (u'javascript_mvc', 0.8

In [85]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9410043358802795),
 (u'backbone', 0.9235603213310242),
 (u'angular', 0.9139069318771362),
 (u'mean_stack', 0.9107990860939026),
 (u'npm', 0.8981935977935791),
 (u'mean', 0.8961061239242554),
 (u'mongodb', 0.8947668075561523),
 (u'bower', 0.89299476146698),
 (u'gulp', 0.8898729085922241),
 (u'full_stack_developer', 0.8898143768310547),
 (u'less', 0.8879895806312561),
 (u'grunt', 0.8874234557151794),
 (u'bootstrap', 0.8813268542289734),
 (u'javascript_mvc', 0.8796409368515015),
 (u'ui_architect', 0.878745436668396),
 (u'express', 0.8783759474754333),
 (u'javascript_developer', 0.8779905438423157),
 (u'loopback', 0.8772644996643066),
 (u'scss', 0.876909077167511),
 (u'meanstack', 0.8767061233520508),
 (u'fullstack_developer', 0.8745248317718506),
 (u'frontend_developer', 0.8743304014205933),
 (u'oo_javascript', 0.8734292984008789),
 (u'frontend', 0.8726052641868591),
 (u'mean_stack_developer', 0.8722931146621704),
 (u'mocha', 0.8718781471252441),
 (u'ionic', 0.86925601959228

In [86]:
re.sub(r'^angular.*$', 'angular', "angular.js")

'angular'